# Import necessary packages

In [4]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import original data and preprocessed data

In [6]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Daily Prediction_Step by Step/sales_train_validation.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [7]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOBBIES')]

In [9]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks_Daily Prediction_Step by Step/List of Product ID according to 4 demand patterns/California/Hobbies/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

# User-defined functions to calculate Metrics and Croston algorithm

In [10]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [11]:
def Croston(ts,extra_periods=1,alpha=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
    # Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = alpha*q + (1-alpha)*p[t]
            f[t+1] = a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Set value for parameters

In [12]:
list_params = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]

In [13]:
list_params

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [14]:
start_train_date = '2014-04-11'
end_train_date = '2016-04-10'
n_pred_days = 14
num_train_needed = 50
validation_training_ratio = 0.95

# Pipeline for Croston

In [15]:
def Croston_output_all_params(sales_pattern, list_params, start_train_date, end_train_date, 
                       n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
    train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]
    
    df_result_all_params = pd.DataFrame()
    for product in list_pattern_py:
        train_first = train_data_first_part[product]
        train_second = train_data_second_part[product]
        for i in list_params:
            predictions = list()
            history = [x for x in train_first[num_train_needed*-1:]]
            for t in range(len(train_second)):
                yhat = Croston(history, extra_periods=1, alpha=i)['Forecast'].iloc[-1]
                predictions.append(yhat)
                history.append(train_second[t])
            df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                            'Actual Data': train_second,
                                            'Forecast': predictions,
                                            'Alpha': [i for count in range(len(train_second))]})
            df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)
            
    return df_result_all_params

In [16]:
start = timer()

pattern_df_result_all_params = Croston_output_all_params(sales_erratic[:200],
                                                      list_params,
                                                      start_train_date,
                                                      end_train_date, 
                                                      n_pred_days,
                                                      num_train_needed,
                                                      validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.22665777101666815 minutes


In [17]:
pattern_df_result_all_params

,Product,Actual Data,Forecast,Alpha
0,HOBBIES_1_004_CA_1_validation,1.0,1.474921,0.1
1,HOBBIES_1_004_CA_1_validation,4.0,1.439869,0.1
2,HOBBIES_1_004_CA_1_validation,2.0,1.633906,0.1
3,HOBBIES_1_004_CA_1_validation,1.0,1.662342,0.1
4,HOBBIES_1_004_CA_1_validation,1.0,1.609721,0.1
...,...,...,...,...
22306,HOBBIES_1_404_CA_4_validation,2.0,5.450678,0.9
22307,HOBBIES_1_404_CA_4_validation,9.0,2.375512,0.9
22308,HOBBIES_1_404_CA_4_validation,14.0,8.331655,0.9
22309,HOBBIES_1_404_CA_4_validation,6.0,13.432660,0.9


In [18]:
pattern_df_result_all_params.to_csv('Croston_Erratic_All_Parameters_Hyperparameter_Tuning_Data.csv')

# Summay all metrics

In [19]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [20]:
start = timer()

df_result_metrics_all_params = pattern_df_result_all_params.groupby(['Product', 'Alpha']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.02013148876666643 minutes


In [21]:
df_result_metrics_all_params

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_004_CA_1_validation,0.1,0.883229,87.7504,102.9179,59.2597,1.422980,1.605773
1,HOBBIES_1_004_CA_1_validation,0.2,0.913740,90.7817,104.8868,63.7266,1.472136,1.654953
2,HOBBIES_1_004_CA_1_validation,0.3,0.938122,93.2042,106.4530,67.4539,1.511419,1.702373
3,HOBBIES_1_004_CA_1_validation,0.4,0.959584,95.3365,107.7216,71.0871,1.545997,1.749943
4,HOBBIES_1_004_CA_1_validation,0.5,0.977967,97.1628,108.6323,74.6439,1.575613,1.799323
...,...,...,...,...,...,...,...,...
598,HOBBIES_1_404_CA_4_validation,0.5,0.849168,78.0887,84.5097,117.9738,6.014940,7.425509
599,HOBBIES_1_404_CA_4_validation,0.6,0.871915,80.1805,85.7154,121.4793,6.176068,7.706379
600,HOBBIES_1_404_CA_4_validation,0.7,0.901502,82.9013,87.7527,125.1749,6.385638,8.009573
601,HOBBIES_1_404_CA_4_validation,0.8,0.929658,85.4905,89.9070,128.3050,6.585080,8.332223


# Check what products has unexpected metrics result

In [22]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
234,HOBBIES_1_189_CA_4_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
235,HOBBIES_1_189_CA_4_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
236,HOBBIES_1_189_CA_4_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
237,HOBBIES_1_189_CA_4_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
238,HOBBIES_1_189_CA_4_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
239,HOBBIES_1_189_CA_4_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
240,HOBBIES_1_189_CA_4_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
241,HOBBIES_1_189_CA_4_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0
242,HOBBIES_1_189_CA_4_validation,0.9,NaN,NaN,NaN,NaN,0.0,0.0
270,HOBBIES_1_234_CA_4_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0


In [23]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
234,HOBBIES_1_189_CA_4_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
235,HOBBIES_1_189_CA_4_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
236,HOBBIES_1_189_CA_4_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
237,HOBBIES_1_189_CA_4_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
238,HOBBIES_1_189_CA_4_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
239,HOBBIES_1_189_CA_4_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
240,HOBBIES_1_189_CA_4_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
241,HOBBIES_1_189_CA_4_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0
242,HOBBIES_1_189_CA_4_validation,0.9,NaN,NaN,NaN,NaN,0.0,0.0
270,HOBBIES_1_234_CA_4_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0


In [24]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
234,HOBBIES_1_189_CA_4_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
235,HOBBIES_1_189_CA_4_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
236,HOBBIES_1_189_CA_4_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
237,HOBBIES_1_189_CA_4_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
238,HOBBIES_1_189_CA_4_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
239,HOBBIES_1_189_CA_4_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
240,HOBBIES_1_189_CA_4_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
241,HOBBIES_1_189_CA_4_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0
242,HOBBIES_1_189_CA_4_validation,0.9,NaN,NaN,NaN,NaN,0.0,0.0
270,HOBBIES_1_234_CA_4_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0


In [25]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
234,HOBBIES_1_189_CA_4_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
235,HOBBIES_1_189_CA_4_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
236,HOBBIES_1_189_CA_4_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
237,HOBBIES_1_189_CA_4_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
238,HOBBIES_1_189_CA_4_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
239,HOBBIES_1_189_CA_4_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
240,HOBBIES_1_189_CA_4_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
241,HOBBIES_1_189_CA_4_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0
242,HOBBIES_1_189_CA_4_validation,0.9,NaN,NaN,NaN,NaN,0.0,0.0
270,HOBBIES_1_234_CA_4_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0


In [26]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [27]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [28]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [29]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 4.477611940298507 %
Percentage of unexpected values of WMAPE is: 4.477611940298507 %
Percentage of unexpected values of SMAPE is: 4.477611940298507 %
Percentage of unexpected values of MAPE is: 4.477611940298507 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [30]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [31]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.1,0.751066,76.680233,84.694539,100.351402,5.202235,7.194870
0.2,0.775599,79.199645,86.059461,105.622895,5.381967,7.374548
0.3,0.798889,81.657798,87.380191,110.289734,5.546521,7.562943
0.4,0.820811,84.004322,88.651719,114.598692,5.699226,7.765398
0.5,0.842420,86.332039,89.921148,118.785411,5.855455,7.987397
0.6,0.864701,88.727284,91.252997,123.059650,6.021448,8.235363
0.7,0.890339,91.486734,92.929097,127.887533,6.210917,8.517844
0.8,0.919126,94.581866,94.957148,133.395792,6.420752,8.848269
0.9,0.952585,98.153112,97.447233,139.795127,6.665918,9.250176


In [32]:
for mts in list_metrics:
    print('The optimum Alpha based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha based on Mean MASE is: 0.1
The optimum Alpha based on Mean WMAPE is: 0.1
The optimum Alpha based on Mean SMAPE is: 0.1
The optimum Alpha based on Mean MAPE is: 0.1
The optimum Alpha based on Mean MAE is: 0.1
The optimum Alpha based on Mean RMSE is: 0.1


### Get MEDIAN metrics value of each Alpha

In [33]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.1,0.723470,74.87470,83.77910,95.03860,4.988893,6.638591
0.2,0.757512,78.13390,85.33170,102.09525,5.145843,6.799115
0.3,0.776062,82.12370,86.99115,109.01175,5.202235,6.992777
0.4,0.796689,85.65470,88.28075,114.41615,5.289468,7.272742
0.5,0.814896,86.32230,90.00445,117.87915,5.437389,7.551746
0.6,0.838360,87.49565,91.15605,121.31230,5.708111,7.753758
0.7,0.860073,90.14520,92.33740,125.25720,6.001554,7.902411
0.8,0.887676,94.01150,94.92090,129.66450,6.290110,8.150779
0.9,0.925091,96.54995,96.76975,132.26790,6.553161,8.454930


In [34]:
for mts in list_metrics:
    print('The optimum Alpha based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha based on Median of MASE is: 0.1
The optimum Alpha based on Median of WMAPE is: 0.1
The optimum Alpha based on Median of SMAPE is: 0.1
The optimum Alpha based on Median of MAPE is: 0.1
The optimum Alpha based on Median of MAE is: 0.1
The optimum Alpha based on Median of RMSE is: 0.1


# Run Croston model after decide best paramaters

In [ ]:
best_alpha = 0.2

In [ ]:
def Croston_test_data(best_alpha, sales_pattern, start_train_date, end_train_date, 
                       n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    
    df_result_best_params = pd.DataFrame()
    for product in list_pattern_py:
        train_product = train_data[product]
        test_product = test_data[product]
        predictions = list()
        history = [x for x in train_product[num_train_needed*-1:]]
        for t in range(len(test_product)):
            yhat = Croston(history, extra_periods=1, alpha=best_alpha)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(test_product[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                        'Actual Data': test_product,
                                        'Forecast': predictions})
        df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)
            
    return df_result_best_params

In [ ]:
start = timer()

pattern_df_result_best_params = Croston_test_data(best_alpha, sales_erratic, start_train_date, end_train_date, n_pred_days, num_train_needed, validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.04617038768333259 minutes


In [ ]:
df_result_final = pattern_df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [ ]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_004_CA_2_validation,1.012094,59.5838,66.1123,100.0527,3.191989,3.652494
1,FOODS_1_018_CA_2_validation,1.168533,82.2813,77.1999,133.1548,3.056162,3.518693
2,FOODS_1_018_CA_3_validation,0.807508,63.2454,65.9768,95.1606,2.484640,3.234042
3,FOODS_1_021_CA_1_validation,0.954901,82.0235,86.1864,159.5221,4.921411,5.965742
4,FOODS_1_021_CA_2_validation,0.913224,63.5159,65.1676,70.7022,2.177689,2.731308
...,...,...,...,...,...,...,...
242,FOODS_3_809_CA_4_validation,0.637197,102.9319,123.6038,34.6375,0.735228,0.916500
243,FOODS_3_811_CA_2_validation,0.973148,71.9526,88.3108,44.1728,3.443445,3.971165
244,FOODS_3_818_CA_2_validation,1.279985,109.3250,134.8358,69.2069,2.264590,2.864551
245,FOODS_3_822_CA_3_validation,0.684961,37.2969,41.2145,28.6306,2.371019,3.029145


In [ ]:
df_result_final.isnull().sum()

Product     0
MASE       19
WMAPE      19
SMAPE      20
MAPE       24
MAE         0
RMSE        0
dtype: int64

In [ ]:
df_result_final.to_csv('Croston_Erratic_Test_Data.csv')